# Motley Fool Random Picks Experiment

We experiment with the idea of randomly choosing 15 stock picks per month and weighting the results based on inverse volatility via Sharpe Ratio

In [1]:
import json
import pandas as pd

import numpy as np
import yfinance as yf

from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from collections import OrderedDict

In [2]:
recommendations = pd.read_json('recommendations.json')

In [3]:
recommendations['date_recommended'] = pd.to_datetime(recommendations['date_recommended'])

In [4]:
recommendations.sort_values(by='date_recommended')

,ticker,date_recommended
243,COST,2002-04-12
242,PYPL EBAY,2002-05-10
241,DIS,2002-06-07
240,ATVI,2002-08-09
239,AMZN,2002-09-06
...,...,...
4,NVTA,2020-02-06
3,DXCM,2020-02-20
2,LK,2020-03-05
1,ZM,2020-03-19


In [5]:
dates = ["2004-01-01", "2020-01-01"]

def generate_dates_array(dates):
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
    total_months = lambda dt: dt.month + 12 * dt.year
    mlist = []
    for tot_m in range(total_months(start)-1, total_months(end)):
        y, m = divmod(tot_m, 12)
        mlist.append(datetime(y, m+1, 1).strftime("%Y-%m-%d"))
    return mlist

months = generate_dates_array(dates)

In [6]:
def inverse_volatility(ticker, end_date, days=252):
    data = yf.Ticker(ticker).history(end=end_date)
    log_ret = np.log(data['Close'] / data['Close'].shift(1))
    return 1 / (np.std(log_ret) * np.sqrt(days))

In [7]:
def calculate_weight_allocations(tickers):
    ratios = []
    final_tickers = []
    pct_allocations = []
    for ticker in tickers:
        if ticker == 'PYPL EBAY' or ticker == 'SINA WB' or ticker == 'BAMXF':
            continue
        iv = inverse_volatility(ticker, month)
        ratios.append(iv)
        final_tickers.append(ticker)
    total_sharpe = sum(ratios)
    for index in range(len(final_tickers)):
        pct_allocations.append(ratios[index] / total_sharpe)
    return final_tickers, pct_allocations

In [8]:
START_AMOUNT = 10000

current_value = START_AMOUNT
current_holdings = []
current_allocations = []

In [9]:
def calculate_returns(amt, ticker, weight_allocation, start_month, end_month):
    data = yf.Ticker(ticker).history(start=start_month, end=end_month)['Close']
    try:
        end = data.iloc[-1]
        start = data.iloc[0]
        pct_return = (end - start)/start
    except Exception:
        pct_return = 0.05

    final_return = (1 + pct_return) * amt * weight_allocation
    return final_return

In [ ]:
history = []
for index, month in enumerate(tqdm(months)):
    if len(current_holdings) != 0:
        new_value = 0
        history.append(current_value)
        for i in range(len(current_holdings)):
            new_value += calculate_returns(current_value, current_holdings[i], current_allocations[i], months[index - 1], month)
        current_value = new_value
    previous_recommendations = recommendations[recommendations['date_recommended'] <= month]['ticker'].unique()
    if len(previous_recommendations) < 15:
        current_holdings, current_allocations = calculate_weight_allocations(previous_recommendations)
    else:
        tickers = np.random.choice(previous_recommendations, 15)
        current_holdings, current_allocations = calculate_weight_allocations(tickers)
    

In [ ]:
current_value